In [1]:
import pandas as pd
import numpy as np
import datetime

pd.options.display.max_rows = 60
pd.options.display.max_columns = 70

In [2]:
h1_res = pd.read_pickle("pickle/h1_res.pick")
h2_res = pd.read_pickle("pickle/h2_res.pick")
h1_dbd = pd.read_pickle("pickle/h1_dbd.pick")
h2_dbd = pd.read_pickle("pickle/h2_dbd.pick")
h1_sim = pd.read_pickle("pickle/h1_sim.pick")
h2_sim = pd.read_pickle("pickle/h2_sim.pick")

In [3]:
h1_dbd.columns

Index(['DOW', 'Occ', 'RoomsSold', 'ADR', 'RoomRev', 'RevPAR', 'NumCancels',
       'Trn_RoomsSold', 'Trn_ADR', 'Trn_RoomRev', 'Grp_RoomsSold', 'Grp_ADR',
       'Grp_RoomRev', 'TrnP_RoomsSold', 'TrnP_ADR', 'TrnP_RoomRev',
       'Cnt_RoomsSold', 'Cnt_ADR', 'Cnt_RoomRev', 'WE', 'WD', 'STLY_Date'],
      dtype='object')

In [4]:
# h1_sim.index = pd.to_datetime(h1_sim.index, format="%Y-%m-%d")
# h2_sim.index = pd.to_datetime(h2_sim.index, format="%Y-%m-%d")

In [5]:
h1_sim.head(2)

,DOW,RoomsOTB,RevOTB,CxlForecast,Trn_RoomsOTB,Trn_CxlProj,Trn_RevOTB,TrnP_RoomsOTB,TrnP_CxlProj,TrnP_RevOTB,Cnt_RoomsOTB,Cnt_CxlProj,Cnt_RevOTB,Grp_RoomsOTB,Grp_CxlProj,Grp_RevOTB,Occ,RevPAR,RemSupply,ADR_OTB,Trn_ADR_OTB,TrnP_ADR_OTB,Grp_ADR_OTB,Cnt_ADR_OTB,WE,WD,STLY_Date,LYA_RoomsSold,LYA_ADR,LYA_RoomRev,LYA_RevPAR,LYA_NumCancels,STLY_OTB,STLY_Rev,STLY_ADR
2017-08-01,Tue,184.0,35239.14,16.0,151.0,14.0,29935.72,13.0,2.0,2488.24,18.0,0.0,2523.52,2.0,0.0,291.66,0.98,188.44,168,191.52,198.25,191.4,145.83,140.20,False,True,2016-08-02,183.0,170.00,31110.89,166.37,81.0,264.0,45992.21,174.212917
2017-08-02,Wed,184.0,35768.37,15.0,150.0,13.0,30567.83,11.0,2.0,2004.24,21.0,0.0,2904.64,2.0,0.0,291.66,0.98,191.27,169,194.39,203.79,182.2,145.83,138.32,False,True,2016-08-03,182.0,173.96,31660.62,169.31,86.0,268.0,47789.33,178.318396


In [6]:
h1_sim.index = pd.to_datetime(h1_sim.index)
h1_pricing = h1_sim[["Trn_RoomsOTB", "Trn_RevOTB", "WD"]].groupby([pd.Grouper(freq='1W'), 'WD']).agg('sum')
h1_pricing = h1_pricing.reset_index().rename(columns={'level_0': 'Date'})
h1_pricing['Date'] = pd.to_datetime(h1_pricing.Date, format='%Y-%m-%d')
h1_pricing["Trn_ADR_OTB"] = round(h1_pricing.Trn_RevOTB / h1_pricing.Trn_RoomsOTB, 2)
h1_pricing.index = h1_pricing.Date
h1_pricing.head(25)

,Date,WD,Trn_RoomsOTB,Trn_RevOTB,Trn_ADR_OTB
Date,,,,,
2017-08-06,2017-08-06,False,269.0,56881.78,211.46
2017-08-06,2017-08-06,True,555.0,115244.03,207.65
2017-08-13,2017-08-13,False,245.0,52122.51,212.74
2017-08-13,2017-08-13,True,594.0,127149.25,214.06
2017-08-20,2017-08-20,False,242.0,52138.31,215.45
2017-08-20,2017-08-20,True,608.0,125568.91,206.53
2017-08-27,2017-08-27,False,213.0,46092.43,216.40
2017-08-27,2017-08-27,True,592.0,127146.36,214.77
2017-09-03,2017-09-03,True,402.0,75773.08,188.49


In [8]:
# dates+pd.DateOffset(weekday=6)

/Users/ewilens/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:702: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  warnings.warn(


In [12]:
import datetime
def apply_rates(row):
    wd = row['WD'] == 1
    date = datetime.datetime.strftime(row.name, format="%Y-%m-%d")
    week_end = datetime.datetime.strftime(row.WeekEndDate, format="%Y-%m-%d")
    mask = (h1_pricing.WD == wd)
    price = h1_pricing[mask].loc[week_end, "Trn_ADR_OTB"]
    return price

h1_sim['SellingPrice'] = h1_sim.apply(apply_rates, axis=1)
    

In [13]:
h1_sim.head()

,DOW,RoomsOTB,RevOTB,CxlForecast,Trn_RoomsOTB,Trn_CxlProj,Trn_RevOTB,TrnP_RoomsOTB,TrnP_CxlProj,TrnP_RevOTB,Cnt_RoomsOTB,Cnt_CxlProj,Cnt_RevOTB,Grp_RoomsOTB,Grp_CxlProj,Grp_RevOTB,Occ,RevPAR,RemSupply,ADR_OTB,Trn_ADR_OTB,TrnP_ADR_OTB,Grp_ADR_OTB,Cnt_ADR_OTB,WE,WD,STLY_Date,LYA_RoomsSold,LYA_ADR,LYA_RoomRev,LYA_RevPAR,LYA_NumCancels,STLY_OTB,STLY_Rev,STLY_ADR,WeekEndDate,SellingPrice
2017-08-01,Tue,184.0,35239.14,16.0,151.0,14.0,29935.72,13.0,2.0,2488.24,18.0,0.0,2523.52,2.0,0.0,291.66,0.98,188.44,168,191.52,198.25,191.40,145.83,140.20,False,True,2016-08-02,183.0,170.00,31110.89,166.37,81.0,264.0,45992.21,174.212917,2017-08-06,207.65
2017-08-02,Wed,184.0,35768.37,15.0,150.0,13.0,30567.83,11.0,2.0,2004.24,21.0,0.0,2904.64,2.0,0.0,291.66,0.98,191.27,169,194.39,203.79,182.20,145.83,138.32,False,True,2016-08-03,182.0,173.96,31660.62,169.31,86.0,268.0,47789.33,178.318396,2017-08-06,207.65
2017-08-03,Thu,176.0,35048.04,14.0,137.0,12.0,29115.71,13.0,2.0,2225.28,23.0,0.0,3253.99,3.0,0.0,453.06,0.94,187.42,162,199.14,212.52,171.18,151.02,141.48,False,True,2016-08-04,183.0,182.11,33326.17,178.21,96.0,279.0,51608.38,184.976272,2017-08-06,207.65
2017-08-04,Fri,179.0,36128.21,14.0,142.0,12.0,30368.95,12.0,2.0,2191.21,22.0,0.0,3114.99,3.0,0.0,453.06,0.96,193.20,165,201.83,213.87,182.60,151.02,141.59,True,False,2016-08-05,183.0,188.02,34408.31,184.00,106.0,289.0,55306.13,191.370692,2017-08-06,211.46
2017-08-05,Sat,171.0,33841.50,15.0,127.0,13.0,26512.83,20.0,2.0,3953.82,22.0,0.0,3086.79,2.0,0.0,288.06,0.91,180.97,156,197.90,208.76,197.69,144.03,140.31,True,False,2016-08-06,180.0,190.06,34211.53,182.95,106.0,286.0,55114.87,192.709336,2017-08-06,211.46


In [14]:
# test above ADR_OTB column

mask = (h1_res.CustomerType=="Transient") & (h1_res.ArrivalDate<='2017-08-04') & (h1_res.CheckoutDate>'2017-08-04') & (h1_res.IsCanceled==0)
test = h1_res[mask]
test.ADR.sum()

31192.449999999997